In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

url = 'https://data.seattle.gov/api/views/65db-xm6k/rows.csv?accessType=DOWNLOAD'
df = pd.read_csv(url, low_memory=False)


In [10]:
# Load the data
df = pd.read_csv("collisions.csv")

C:\Users\shwet\AppData\Local\Temp\ipykernel_11816\2459775208.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("collisions.csv")


In [11]:
df.head()

,X,Y,OBJECTID,INCKEY,COLDETKEY,ADDRTYPE,COLLISIONTYPE,EXCEPTRSNCODE,EXCEPTRSNDESC,FATALITIES,...,ADDDTTM,MODBY,MODDTTM,ST_PARTCPNT_TYPE,ST_PARTCPNT_TYPE_DESC,ST_AGE,ST_CITED,ST_VEH_TYPE_CD,ST_VEH_TYPE_DESC,ST_INJRY_CLSS_DESC
0,1.271757e+06,202831.432061,1,347106,348606,Intersection,Pedestrian,,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.259163e+06,196658.337780,2,353843,355303,Block,Rear Ended,,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.269312e+06,204946.524287,3,345966,347466,Intersection,Left Turn,,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.267757e+06,271122.941803,4,353658,355118,Block,Parked Car,,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.281388e+06,269060.231034,5,345924,347424,Intersection,NaN,,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Select relevant features
features = ['ADDRTYPE', 'COLLISIONTYPE', 'JUNCTIONTYPE', 'WEATHER', 'ROADCOND', 'LIGHTCOND']


In [13]:
# Preparing the data by selecting the features we want to use as predictors and the target variable:
# Select features
features = ['ADDRTYPE', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'SPEEDING', 'UNDERINFL']

# Select target variable
target = 'SEVERITYCODE'


In [14]:
# Make sure all the columns in features and target exist in df
valid_cols = set(features + [target]).issubset(set(df.columns))
if not valid_cols:
    raise ValueError('One or more columns in features or target do not exist in df')


In [15]:
# Create data subset with only selected features and target variable
data = df[features + [target]].dropna()

In [16]:
# Split features into categorical and numerical
cat_features = ['ADDRTYPE', 'COLLISIONTYPE', 'JUNCTIONTYPE', 'WEATHER', 'ROADCOND', 'LIGHTCOND']
num_features = []


In [18]:
# Make sure all the categorical features in cat_features exist in df
valid_cols = set(cat_features).issubset(set(df.columns))
if not valid_cols:
    raise ValueError('One or more columns in cat_features do not exist in df')


In [20]:
print(df.columns)

Index(['X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'ADDRTYPE',
       'COLLISIONTYPE', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC', 'FATALITIES',
       'INATTENTIONIND', 'INCDATE', 'INCDTTM', 'INJURIES', 'INTKEY',
       'JUNCTIONTYPE', 'LIGHTCOND', 'LOCATION', 'PEDCOUNT', 'PEDCYLCOUNT',
       'PEDROWNOTGRNT', 'PERSONCOUNT', 'REPORTNO', 'ROADCOND', 'SDOT_COLCODE',
       'SDOT_COLDESC', 'SDOTCOLNUM', 'SERIOUSINJURIES', 'SEVERITYCODE',
       'SEVERITYDESC', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC', 'STATUS',
       'UNDERINFL', 'VEHCOUNT', 'WEATHER', 'SEGLANEKEY', 'CROSSWALKKEY',
       'HITPARKEDCAR', 'SOURCE', 'SOURCEDESC', 'ADDBY', 'ADDDTTM', 'MODBY',
       'MODDTTM', 'ST_PARTCPNT_TYPE', 'ST_PARTCPNT_TYPE_DESC', 'ST_AGE',
       'ST_CITED', 'ST_VEH_TYPE_CD', 'ST_VEH_TYPE_DESC', 'ST_INJRY_CLSS_DESC'],
      dtype='object')


In [21]:
from sklearn.compose import make_column_selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Select the categorical and numerical features separately
num_features = make_column_selector(dtype_include=['int64', 'float64'])(df[features])
cat_features = make_column_selector(dtype_exclude=['int64', 'float64'])(df[features])

# Create transformers for numerical and categorical features
num_transformer = StandardScaler()
cat_transformer = OneHotEncoder(drop='first')

# Use ColumnTransformer to apply transformers to selected features
preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])

# Fit the preprocessor and transform the selected features of the df DataFrame
X = preprocessor.fit_transform(df[features])


In [22]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.2, random_state=42)


In [23]:
# Define the preprocessor to use for feature preprocessing
num_features = make_column_selector(dtype_include=['int64', 'float64'])(X_train)
cat_features = make_column_selector(dtype_exclude=['int64', 'float64'])(X_train)
num_transformer = StandardScaler()
cat_transformer = OneHotEncoder(drop='first')
preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])


In [24]:
# Preprocess the features using the preprocessor
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)


In [31]:
# Train a logistic regression model
logreg = LogisticRegression(random_state=42)
logreg.fit(X_train_preprocessed, y_train)
y_pred_logreg = logreg.predict(X_test_preprocessed)
acc_logreg = accuracy_score(y_test, y_pred_logreg)
print('Accuracy of logistic regression model: {:.2f}'.format(acc_logreg))


Accuracy of logistic regression model: 0.62


In [32]:
# Train a random forest model
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_preprocessed, y_train)
y_pred_rf = rf.predict(X_test_preprocessed)
acc_rf = accuracy_score(y_test, y_pred_rf)
print('Accuracy of random forest model: {:.2f}'.format(acc_rf))

Accuracy of random forest model: 0.60


In [33]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters to tune
param_grid = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}

# Create a logistic regression model
logreg = LogisticRegression(random_state=42)

# Create a GridSearchCV object with 5-fold cross-validation
grid_logreg = GridSearchCV(logreg, param_grid=param_grid, cv=5, n_jobs=-1)

# Fit the GridSearchCV object to the preprocessed training data
grid_logreg.fit(X_train_preprocessed, y_train)

# Print the best hyperparameters and the corresponding mean cross-validation score
print('Best hyperparameters:', grid_logreg.best_params_)
print('Mean cross-validation score:', grid_logreg.best_score_)


Best hyperparameters: {'C': 0.01, 'penalty': 'l2'}
Mean cross-validation score: 0.589779521076642


In [34]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters to tune
param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [5, 10, None]}

# Create a random forest model
rf = RandomForestClassifier(random_state=42)

# Create a GridSearchCV object with 5-fold cross-validation
grid_rf = GridSearchCV(rf, param_grid=param_grid, cv=5, n_jobs=-1)

# Fit the GridSearchCV object to the preprocessed training data
grid_rf.fit(X_train_preprocessed, y_train)

# Print the best hyperparameters and the corresponding mean cross-validation score
print('Best hyperparameters:', grid_rf.best_params_)
print('Mean cross-validation score:', grid_rf.best_score_)


Best hyperparameters: {'max_depth': 5, 'n_estimators': 50}
Mean cross-validation score: 0.5912964405632231


Note that in both cases, we define a dictionary of hyperparameters to tune, create a model object, create a GridSearchCV object with the hyperparameters and cross-validation settings, and fit the GridSearchCV object to the preprocessed training data. Finally, we print the best hyperparameters and the mean cross-validation score, which gives us an estimate of the performance of the model on new, unseen data.